# Social Graphs 2021 - Assignment 2
## Quick disclaimer
The assignment description could hint at not implementing some of the previous exercises, but just use the files already downloaded and processed. This seemed to make the exercises and explanations a little incoherrent, so we have included the code for the API-calls and other misc stuff. We have tried to collapse the cell blocks, so this is not annoying for the reader, but if they do appear, sorry for the inconvinience.

In [1]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import wordcloud
import nltk
import networkx as nx

import re
import json
import urllib

import os

In [2]:
# Styles and data. If this cell block is not already hidden/minimized, feel free to do so, as it only contains the styles for the plots, 
# and the characters.csv in written form, as it allows for reruns of the code, if needed.
DTU_COLORS = dict(
    dtured      = "#990000",
    blue        = "#2F3EEA",
    brightgreen = "#1FD082",
    navyblue    = "#030F4F"
)

# 0. Building the network
As this process is a two step of finding names, and then their pages, we have shuffled the explanation of strategy and regular expressions

## Strategy for extracting hyperlinks from the wiki-pages
### Downloading
First we download the three wiki pages containing the ally, enemy and boss characters. From this we use a regular expression to find all character names.
### Regex
First the boss page has an edge case, where the location of the boss is included in the same format, as the name of the boss. To remove this, we use ```{{Small.*?''}}``` to find the locations and remove them.

When these are removed we use ```({{Term\|BotW\|(.*?)\|link[\|]*.*?}})``` to find the names of each character. All names follow the same convention of `{{Term|BotW|NAME OF CHARACTER|link|MAYBE MORE STUFF}}` occasionally followed by further information. This is solved by using `*.*?` which lazily finds additional terms. `*?` makes sure, that this is only done until the following character `}` is found.

## Strategy for finding gender and race
As we already have the character pages, we do a regex expression for gender, and for race.

* Race: `{{Infobox.*?\|(?:race=|species=)\s(.*?)(?:\n|<ref).*?(?:\|gender=\s`**(.*?)**`(?:\n|<ref)|).*?}}`. 

For each character page we use this expression. All metainformation, including the races, is contained in an Infobox, explaining the beginning and end of the expressions. We use capture groups to allow for or-statements signified by `(|)` the |-symbol, as this cannot be written outside of a capture group. As whether the keyword is `race` or `species` is not important `(?:)` is used, to make the capture group not return anything. The same convention is followed throughout the expression, to include different types of definitions. Lastly the small group `(.*?)` is used to find the race of the character. Characters might have multiple races, in which case only the first is chosen.

* Gender: `.*(Male|Female).*`. 

Genders are only included inside of infoboxes, which we found in the previous expression. Using the extraction from the previous expression this expression is now used. This expression is only run for these infoboxes. The expression is only used on these, searching for males or females, where the `|`-symbol signifies an or-statement. For genders outside of male or female, the gender `unknown` is assigned to the character. 

From here we do an API-call using the character names to get each characters content. Then we use the regular the same set of regular expressions for each character extracting the links to other characters.

## 

# Regex strategy
We do two different regex calls. First we find the characters in the 

The main regular expression is to find every link o a character. We know they have the shape `{{Term|BotW|name of character|link|maybe more stuff}}`. Unfortionately in the texts locations follow the same pattern. This is first removed with the expression `{{Small.*?'}}`, to filter out the locations.